# TPSC : Self energy


Nearest neighbour $t$, next nearest neighbour $t'=0.175$, and next next nearest neighbour hopping $t''=0.05$, interaction $U = 5.75t$, and inverse temperature $\beta = 20$.

In [ ]:
beta = 20.
mu = 0.0

t = 1.0
tp = 0.175 * t
tpp = 0.05 * t

U = 5.75 * t

n_k = (8, 8, 1) 
n_w = 1024 * 4

In [ ]:
import numpy as np
from pytriqs.lattice.tight_binding import TBLattice

I = np.eye(2)

H_0 = TBLattice(
    units = [(1, 0, 0), (0, 1, 0)],
    hopping = {
        (+1, 0): -t * I,  
        (-1, 0): -t * I, 
        ( 0,+1): -t * I, 
        ( 0,-1): -t * I,
        # next nearest neighbour
        (+1, +1): -tp * I,  
        (+1, -1): -tp * I,  
        (-1, +1): -tp * I,  
        (-1, -1): -tp * I,  
        # next next nearest neighbour
        (+2, 0): -tpp * I,  
        (-2, 0): -tpp * I, 
        ( 0,+2): -tpp * I, 
        ( 0,-2): -tpp * I,
        },
    orbital_positions = [(0,0,0)]*2,
    orbital_names = ['up', 'do'],
    )

In [ ]:
kmesh = H_0.get_kmesh(n_k)
print 'kmesh =\n', kmesh
rmesh = H_0.get_rmesh(n_k)
print 'rmesh =\n', rmesh

e_k = H_0.on_gf_mesh_brillouin_zone(n_k)
print 'e_k =\n', e_k

In [ ]:
# Fixme move 2d plot to library

from k_space_viz import extend_data_on_boundary

data_k, k_vec, (kx, ky, kz) = extend_data_on_boundary(e_k[0, 0].data, n_k)
data_k = data_k[:, :, 0].real
extent_k = np.array([kx.min(), kx.max(), ky.min(), ky.max()])

from pytriqs.plot.mpl_interface import plt

plt.figure(figsize=(3.25*3, 8))
plt.title(r'Dispersion $\epsilon_k$ and fermi surface contour $\epsilon_k = 0$')
plt.contour(kx, ky, data_k, levels=[0.])
plt.imshow(data_k, cmap=plt.get_cmap('RdBu_r'), origin='lower', 
           vmin=data_k.min(), vmax=data_k.max(),
           extent=extent_k)
plt.xlabel(r'$k_x/(2\pi)$')
plt.ylabel(r'$k_y/(2\pi)$')
plt.colorbar(); plt.tight_layout()
plt.savefig('figure_e_k.pdf')
plt.show()

In [ ]:
from k_space_viz import k_space_path

G = np.array([0.0, 0.0, 0.0])
X = np.array([0.5, 0.5, 0.0])
M = np.array([0.5, 0.0, 0.0])

paths = [(G, X), (X, M), (M, G)]

k_path_vecs, k_plot, K_plot = k_space_path(paths, num=100)

from k_space_viz import get_rel_k_interpolator

interp = get_rel_k_interpolator(
    e_k[0, 0].data, kmesh, H_0.bz, n_k,
    extend_boundary=True, interpolator='linear2D')
e_plot = interp(k_path_vecs[:, :2])

from pytriqs.plot.mpl_interface import plt

plt.figure(figsize=(3.25*3, 8))
plt.title(r'Dispersion $\epsilon_k$ along the high-symmetry path $\Gamma$-$X$-$M$-$\Gamma$')
plt.plot(k_plot, e_plot.real)

plt.grid(); plt.axes().set_xticks(K_plot)
plt.xlim([K_plot.min(), K_plot.max()])
plt.axes().set_xticklabels([r'$\Gamma$',r'$X$',r'$M$',r'$\Gamma$'])
plt.ylabel(r'$\epsilon_k$'); plt.tight_layout()
plt.savefig('figure_e_k_bandpath.pdf')
plt.show()

## TPSC calculation


In [ ]:
print 'g0_wk'
g0_wk, g0_wr, g0_tr = get_g0(e_k, beta, mu, n_w, kmesh, rmesh)
n = -g0_tr[Idx(0), Idx(0, 0, 0)].sum().real
print 'n =', n

In [ ]:
print 'chi0_wk'
chi0_tr, chi0_wr, chi0_wk = get_chi0(g0_tr, n_w, kmesh, rmesh)

print 'tpsc self cons'
Usp, Uch, docc = solve_Usp_and_Uch(chi0_wk, U, n, Usp0=Usp, Uch0=Uch)
print 'Usp, Uch, docc =', Usp, Uch, docc

In [ ]:
print 'chi_sp_wk, chi_ch_wk'
chi_sp_wk = chi_wk_from_U_and_chi0_wk(chi0_wk, +0.5*Usp)
chi_ch_wk = chi_wk_from_U_and_chi0_wk(chi0_wk, -0.5*Uch)

In [ ]:
# todo: remove when we have the new tail

def fix_tr_tail(g_tr, r, gf_flag=True):
    g_t = g_tr[:, r]
    g_t.singularity = TailGf(g_t.target_shape)
    if gf_flag and np.linalg.norm(r.value) < 1e-9: g_t.singularity[1] = I
    return g_t

In [ ]:
target_shape = [1, 1]
wmesh = chi0_wk.mesh.components[0]
tmesh = chi0_tr.mesh.components[0]

chi_sp_wr = Gf(mesh=MeshProduct(wmesh, rmesh), target_shape=target_shape)
chi_sp_tr = Gf(mesh=MeshProduct(tmesh, rmesh), target_shape=target_shape)

chi_ch_wr = Gf(mesh=MeshProduct(wmesh, rmesh), target_shape=target_shape)
chi_ch_tr = Gf(mesh=MeshProduct(tmesh, rmesh), target_shape=target_shape)

print 'k -> r'
for w in wmesh: 
    chi_sp_wr[w, :] = InverseFourier(chi_sp_wk[w, :])
    chi_ch_wr[w, :] = InverseFourier(chi_ch_wk[w, :])

print 'w -> t'
for r in rmesh: 
    chi_sp_tr[:, r] = InverseFourier(fix_tr_tail(chi_sp_wr, r, gf_flag=False)) # Q: SHOULD WE FIX TAIL HERE??!!
    chi_ch_tr[:, r] = InverseFourier(fix_tr_tail(chi_ch_wr, r, gf_flag=False))

In [ ]:
plt.figure()
oplotr(chi_sp_tr[:, Idx(0, 0, 0)])
plt.figure()
oplotr(chi_sp_tr[:, Idx(1, 0, 0)])

In [ ]:
plt.figure()
oplotr(chi_ch_tr[:, Idx(0, 0, 0)])
plt.figure()
oplotr(chi_ch_tr[:, Idx(1, 0, 0)])

In [ ]:
# compute self energy

g_chi_sp_tr = bubble(g0_tr[0, 0], chi_sp_tr)
g_chi_ch_tr = bubble(g0_tr[0, 0], chi_ch_tr)

sigma_tr = U/8. * ( 3.*Usp*g_chi_sp_tr + Uch*g_chi_ch_tr )

print sigma_tr

In [ ]:
plt.figure()
oplotr(sigma_tr[:, Idx(0, 0, 0)])
plt.figure()
oplotr(sigma_tr[:, Idx(0, 0, 0)])
plt.xlim([19.5, 20.1])
plt.figure()
oplotr(sigma_tr[:, Idx(1, 0, 0)])

In [ ]:
target_shape = [1, 1]

sigma_wr = Gf(mesh=MeshProduct(wmesh, rmesh), target_shape=target_shape)
sigma_wk = Gf(mesh=MeshProduct(wmesh, kmesh), target_shape=target_shape)

for r in rmesh: sigma_wr[:, r] = Fourier(fix_tr_tail(sigma_tr, r, gf_flag=False))
for w in wmesh: sigma_wk[w, :] = Fourier(sigma_wr[w, :])

In [ ]:
# add hartree term
#for k in kmesh: sigma_wk[:, k] += U * 0.5 * n * I

In [ ]:
# Analytic continuation of Sigma_wk to w = 0 + i*epsilon

from pytriqs.gf import MeshReFreq

rwmesh = MeshReFreq(omega_min=-1., omega_max=1., n_max=10)
sigma_rwk = Gf(mesh=MeshProduct(rwmesh, kmesh), target_shape=[1, 1])

for k in kmesh:
    sigma_rw = sigma_rwk[:, k]
    sigma_rw.set_from_pade(sigma_wk[:, k], n_points=20)

In [ ]:
oplot(sigma_wk[:, Idx(0, 0, 0)])

In [ ]:
oplot(sigma_rwk[:, Idx(0, 0, 0)])

In [ ]:
# Plot G(w=0, k)
g0_rwk = Gf(mesh=MeshProduct(rwmesh, kmesh), target_shape=[1, 1])
g_rwk = Gf(mesh=MeshProduct(rwmesh, kmesh), target_shape=[1, 1])

for k in kmesh:
    eps = 5.e-1j
    g_rwk[:, k] << inverse(iOmega_n + eps - e_k[k][0, 0] - sigma_rwk[:, k])
    g0_rwk[:, k] << inverse(iOmega_n + eps - e_k[k][0, 0])    

In [ ]:
oplot(g0_rwk[:, Idx(0, 0, 0)])

In [ ]:
oplot(g_rwk[:, Idx(0, 0, 0)])

In [ ]:
g_k = g_rwk[Idx(5), :]
g0_k = g0_rwk[Idx(5), :]

In [ ]:
# Fixme move 2d plot to library

from k_space_viz import extend_data_on_boundary

data_k, k_vec, (kx, ky, kz) = extend_data_on_boundary(g_k.data, n_k)
data_k = - data_k[:, :, 0].imag / np.pi
extent_k = np.array([kx.min(), kx.max(), ky.min(), ky.max()])

from pytriqs.plot.mpl_interface import plt

plt.figure(figsize=(3.25*3, 8))
plt.imshow(data_k, cmap=plt.get_cmap('RdBu_r'), origin='lower', 
           vmin=data_k.min(), vmax=data_k.max(),
           extent=extent_k)
plt.xlabel(r'$k_x/(2\pi)$')
plt.ylabel(r'$k_y/(2\pi)$')
plt.colorbar(); plt.tight_layout()

In [ ]:
# Fixme move 2d plot to library

from k_space_viz import extend_data_on_boundary

data_k, k_vec, (kx, ky, kz) = extend_data_on_boundary(g0_k.data, n_k)
data_k = - data_k[:, :, 0].imag / np.pi
extent_k = np.array([kx.min(), kx.max(), ky.min(), ky.max()])

from pytriqs.plot.mpl_interface import plt

plt.figure(figsize=(3.25*3, 8))
plt.imshow(data_k, cmap=plt.get_cmap('RdBu_r'), origin='lower', 
           vmin=data_k.min(), vmax=data_k.max(),
           extent=extent_k)
plt.xlabel(r'$k_x/(2\pi)$')
plt.ylabel(r'$k_y/(2\pi)$')
plt.colorbar(); plt.tight_layout()